# Introduction
For creating the sentence embedding, we are going to use nlu library. Details can be found [here](https://nlu.johnsnowlabs.com/). 
First of all we are going to install required modules

In [ ]:
import os
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install nlu  pyspark==2.4.7 > /dev/null   

import nlu

# Downloading data set
Dataset is taken from stoackoverflow questions

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Download the dataset 
! wget -N https://ckl-it.de/wp-content/uploads/2020/11/60kstackoverflow.csv -P /tmp
# Load dataset to Pandas
df = pd.read_csv('/tmp/60kstackoverflow.csv')
max_r = 5000
df = df.iloc[0:max_r]
df

--2021-07-18 00:29:41--  https://ckl-it.de/wp-content/uploads/2020/11/60kstackoverflow.csv
Resolving ckl-it.de (ckl-it.de)... 217.160.0.108, 2001:8d8:100f:f000::209
Connecting to ckl-it.de (ckl-it.de)|217.160.0.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50356825 (48M) [text/csv]
Saving to: ‘/tmp/60kstackoverflow.csv’

60kstackoverflow.cs 100%[===================>]  48.02M  21.5MB/s    in 2.2s    

2021-07-18 00:29:44 (21.5 MB/s) - ‘/tmp/60kstackoverflow.csv’ saved [50356825/50356825]



,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ
...,...,...,...,...,...,...
4995,36742008,is it possible to use JavaScriptSpellCheck in ...,<p>I would like to perform Spell Checking on t...,<javascript><html><jsp>,2016-04-20 11:11:19,LQ_CLOSE
4996,36743857,"Map<String, Integer> foo - how do I get the va...",<p>I have a method:</p>\n\n<pre><code> public...,<java><dictionary><hashmap>,2016-04-20 12:29:50,LQ_CLOSE
4997,36744886,Facebook SDK iOS - User photos doesn't retriev...,<p>I am facing a weird bug with Facebook SDK. ...,<ios><objective-c><facebook><facebook-graph-ap...,2016-04-20 13:09:41,LQ_CLOSE
4998,36746382,Validate user input using seperate function,<p>How do I get this to not return 'brea outsi...,<python><python-3.x><python-3.5>,2016-04-20 14:07:03,LQ_CLOSE


# Bert sentence embedding

In [ ]:
pipe = nlu.load('embed_sentence.bert')
# pipe = nlu.load('en.embed_sentence.bert_large_cased') # if you have some time and RAM try a big BERT model!
predictions = pipe.predict(df.Title, output_level='document')
predictions

sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


,text,document,sentence_embedding_bert
0,Java: Repeat Task Every Random Seconds,Java: Repeat Task Every Random Seconds,"[-2.009953498840332, 0.08406741172075272, -0.0..."
1,Why are Java Optionals immutable?,Why are Java Optionals immutable?,"[-0.9489684700965881, -0.04183836281299591, -0..."
2,Text Overlay Image with Darkened Opacity React...,Text Overlay Image with Darkened Opacity React...,"[-0.7274762392044067, -0.8136711716651917, -0...."
3,Why ternary operator in swift is so picky?,Why ternary operator in swift is so picky?,"[-1.0510447025299072, 0.16469576954841614, -0...."
4,hide/show fab with scale animation,hide/show fab with scale animation,"[-0.801064133644104, -0.08930190652608871, -0...."
...,...,...,...
4995,is it possible to use JavaScriptSpellCheck in ...,is it possible to use JavaScriptSpellCheck in ...,"[-0.28274181485176086, 0.9793228507041931, 0.2..."
4996,"Map<String, Integer> foo - how do I get the va...","Map<String, Integer> foo - how do I get the va...","[-0.7471789121627808, 0.14025633037090302, 0.1..."
4997,Facebook SDK iOS - User photos doesn't retriev...,Facebook SDK iOS - User photos doesn't retriev...,"[0.2970831096172333, 0.5095561146736145, -0.35..."
4998,Validate user input using seperate function,Validate user input using seperate function,"[-0.9171112775802612, 0.5364910960197449, -0.5..."


## Writing embeddings to file

In [ ]:
savedAtGoogleDrive=True
if savedAtGoogleDrive==True:
  from google.colab import drive
  drive.mount('/content/gdrive')
  filename_input="gdrive/MyDrive/bert_sentenceEmbedding.txt"
  f = open(filename_input, "w")
else:
  filename_input="bert_sentenceEmbedding.txt"
  f = open(filename_input, "w")
for i in range(len(predictions["document"])):
  f.write(f" <pad2a>{predictions.document[i]}<pad2a> ")
  pred=predictions.sentence_embedding_bert[i]
  for j in range(len(pred)):
    f.write(f"{pred[j]} ")
  f.write('\n')
f.close()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Introducing weights

In [ ]:
weights_reconstruction=1e3;
weights_PSL=50;
weights_ASL=5; 

## Datahandler class

In [ ]:
import numpy as np
import logging
from sklearn.datasets import make_blobs
#logging.basicConfig(level=logging.INFO)
logging.basicConfig(level=logging.DEBUG)

class DataHandler:
  def  __init__(self):
    pass
    
  def loadData(self,filename):
    logging.info("Loading data from " + filename )
    #limit = 1000 ## for debnugging. TODO: remove this
    lines = open(filename).readlines()#[:limit] 
    self.words = []
    self.data=[]
    print(f"Total Lines in the file:{len(lines)}")
    for line in lines:
      # tokens = line.strip().split()
      tokens = line.strip().split("<pad2a>")
      tokens_sent=tokens[1]
      tokens_val=tokens[2]
      # tokens_sent=tokens_sent.split()
      tokens_val=tokens_val.split()
      try:
        elements=[[float(v)] for v in tokens_val]
      except:
        print('#############################################')
        print(tokens_val)
        print(np.shape(tokens))
        print(tokens)
        break
      self.data.append(elements)
      self.words.append([tokens_sent])
    self.data=np.asarray(self.data)
    self.data=self.data.squeeze(axis=2)
    print(f'shape of vector space: {np.shape(self.data)}')
    print(f'shape of sentences: {np.shape(self.words)}')

    # print(self.words[0])
    # print(self.data[0])
    logging.info("Loaded data. #shape = " + str(self.data.shape) )
    logging.info(" #Sentences = %d " %(len(self.words)) )
    self.data_size = self.data.shape[0]
    self.inp_dim = self.data.shape[1]
    self.original_data = self.data[:]
    logging.debug("original_data[0][0:5] = " + str(self.original_data[0][0:5]))

  def getWordsList(self):
    return self.words
  
    

  def getDataShape(self):
    return self.data.shape

  def resetDataOrder(self):
    self.data = self.original_data[:]
    logging.debug("original_data[0][0:5] = " + str(self.original_data[0][0:5]))

  def getNumberOfBatches(self, batch_size):
    return int(( self.data_size + batch_size - 1 ) / batch_size)

  def getBatch(self, i, batch_size, noise_level, denoising):
    batch_y = self.data[i*batch_size:min((i+1)*batch_size, self.data_size)]
    batch_x = batch_y
    if denoising:
      batch_x = batch_y + get_noise_features(batch_y.shape[0], self.inp_dim, noise_level)
    return batch_x, batch_y

  def shuffleTrain(self):
    indices = np.arange(self.data_size)
    np.random.shuffle(indices)
    self.data = self.data[indices]


############################################

############################################
#### data handler class finished############

## Model

In [ ]:
# Model
import torch
from torch import nn
import logging
logging.basicConfig(level=logging.INFO)


class SPINEModel(torch.nn.Module):

	def __init__(self, params):
		super(SPINEModel, self).__init__()
		
		# params
		self.inp_dim = params['inp_dim']
		self.hdim = params['hdim']
		self.noise_level = params['noise_level']
		self.getReconstructionLoss = nn.MSELoss()
		self.rho_star = 1.0 - params['sparsity']
		
		# autoencoder
		logging.info("Building model ")
		self.linear1 = nn.Linear(self.inp_dim, self.hdim)
		self.linear2 = nn.Linear(self.hdim, self.inp_dim)
		

	def forward(self, batch_x, batch_y):
		
		# forward
		batch_size = batch_x.size(0)
		linear1_out = self.linear1(batch_x)
    # capped relu
		h = linear1_out.clamp(min=0, max=1) 
		out = self.linear2(h)

		# different terms of the loss
		reconstruction_loss = self.getReconstructionLoss(out, batch_y) # reconstruction loss
		psl_loss = self._getPSLLoss(h, batch_size) 		# partial sparsity loss
		asl_loss = self._getASLLoss(h)    	# average sparsity loss
		total_loss = weights_reconstruction*reconstruction_loss + \
    weights_PSL* psl_loss + weights_ASL*asl_loss
		
		return out, h, total_loss, [reconstruction_loss,psl_loss, asl_loss]


	def _getPSLLoss(self,h, batch_size):
		return torch.sum(h*(1-h))/ (batch_size * self.hdim)


	def _getASLLoss(self, h):
		temp = torch.mean(h, dim=0) - self.rho_star
		temp = temp.clamp(min=0)
		return torch.sum(temp * temp) / self.hdim



DEBUG:py4j.java_gateway:Command to send: m
d
o174
e

DEBUG:py4j.java_gateway:Answer received: !yv
DEBUG:py4j.java_gateway:Command to send: m
d
o295
e

DEBUG:py4j.java_gateway:Answer received: !yv


## Some helper functions

In [ ]:
def compute_sparsity(X):
	non_zeros = 1. * np.count_nonzero(X)
	total = X.size
	sparsity = 100. * (1 - (non_zeros)/total)
	return sparsity

def dump_vectors(X, outfile, words):
  print ("shape", X.shape)
  assert len(X) == len(words) #TODO print error statement
  fw = open(outfile, 'w')
  for i in range(len(words)):
    sent=words[i][:]
    sent=str(sent)
    sent=sent.replace("[","")
    sent=sent.replace("'","")
    sent=sent.replace("]","")
    # print(np.shape(sent[0]))
    # print(sent[0])  
    fw.write("<pad2a>"+ sent+"<pad2a>" )
    for j in X[i,:]:
      fw.write(str(j) + " ")
    fw.write("\n")
  fw.close()

def get_noise_features(n_samples, n_features, noise_amount):
	noise_x,  _ =  make_blobs(n_samples=n_samples, n_features=n_features, 
			cluster_std=noise_amount,
			centers=np.array([np.zeros(n_features)]))
	return noise_x


In [ ]:
import torch
from torch import nn
import argparse
from random import shuffle
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

class Solver:

	def __init__(self, params):

		# Build data handler
		self.data_handler = DataHandler()
		self.data_handler.loadData(params['input'])
		params['inp_dim'] = self.data_handler.getDataShape()[1]
		logging.info("="*41)


		# Build model
		self.model = SPINEModel(params)
		self.dtype = torch.FloatTensor
		use_cuda = torch.cuda.is_available()
		if use_cuda:
			self.model.cuda()
			self.dtype = torch.cuda.FloatTensor
		self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.1)
		logging.info("="*41)


	def train(self, params):
		num_epochs, batch_size = params['num_epochs'], params['batch_size'],
		optimizer = self.optimizer
		dtype = self.dtype
		for iteration in range(num_epochs):
			self.data_handler.shuffleTrain()
			num_batches = self.data_handler.getNumberOfBatches(batch_size)
			epoch_losses = np.zeros(4) # rl, asl, psl, total
			for batch_idx in range(num_batches):
				optimizer.zero_grad()
				batch_x, batch_y = self.data_handler.getBatch(batch_idx, batch_size, params['noise_level'], params['denoising'] )
				batch_x = torch.from_numpy(batch_x).type(dtype)
				batch_y = torch.from_numpy(batch_y).type(dtype)
				out, h, loss, loss_terms = self.model(batch_x, batch_y)
				reconstruction_loss, psl_loss, asl_loss = loss_terms
				loss.backward()
				optimizer.step()
				epoch_losses[0]+=reconstruction_loss.item()
				epoch_losses[1]+=asl_loss.item()
				epoch_losses[2]+=psl_loss.item()
				epoch_losses[3]+=loss.item()
			print("After epoch %r, Reconstruction Loss = %.4f, ASL = %.4f,"\
						"PSL = %.4f, and total = %.4f"
						%(iteration+1, epoch_losses[0], epoch_losses[1], epoch_losses[2], epoch_losses[3]) )
			logging.info("After epoch %r, Sparsity = %.1f"
						%(iteration+1, compute_sparsity(h.cpu().data.numpy())))
				#break
			#break

	def getSpineEmbeddings(self, batch_size, params):
		ret = []
		self.data_handler.resetDataOrder()
		num_batches = self.data_handler.getNumberOfBatches(batch_size)
		for batch_idx in range(num_batches):
			batch_x, batch_y = self.data_handler.getBatch(batch_idx, batch_size, params['noise_level'], params['denoising'] )
			batch_x = torch.from_numpy(batch_x).type(self.dtype)
			batch_y = torch.from_numpy(batch_y).type(self.dtype)
			_, h, _, _ = self.model(batch_x, batch_y)
			ret.extend(h.cpu().data.numpy())
		return np.array(ret)

	def getWordsList(self):
		return self.data_handler.getWordsList()



In [ ]:
#########################################################
# main program
def main(hdim=1000,
                  denoising=False,
                  noise_level=0.2,
                  num_epochs=100,
                  batch_size=64,
                  sparsity=0.85,
                  fileName='word2vec_original_15k_300d_train.txt'):
  params=dict()
  # params['inp_dim']=2000
  params['hdim']=hdim
  params['noise_level']=noise_level
  params['denoising']=denoising
  params['sparsity']=sparsity
  params['num_epochs']=num_epochs
  params['batch_size']=batch_size
  params['input']=fileName
  params['output']=fileName
  solver = Solver(params)
  solver.train( params )
  # dumping the final vectors
  logging.info("Dumping the final SPine embeddings")
  output_path = params['output'] + "_SPINE.spine"
  final_batch_size = 512
  spine_embeddings = solver.getSpineEmbeddings(final_batch_size, params)
  dump_vectors(spine_embeddings, output_path, solver.getWordsList())
  

In [ ]:
weights_reconstruction=1;
weights_PSL=0.8;
weights_ASL=1;
epochs=300
if savedAtGoogleDrive==True:
  drive_path='gdrive/MyDrive/'
else:
  drive_path=""
filename="bert_sentenceEmbedding.txt"
filename=drive_path+filename
hdim=1000
denoising=True
noise_level=0.6
batch_size=512
sparsity=0.85
main(num_epochs=epochs,denoising=denoising,noise_level=noise_level,
     hdim=hdim,batch_size=batch_size,sparsity=sparsity,
     fileName=filename)

INFO:root:Loading data from gdrive/MyDrive/bert_sentenceEmbedding.txt


Total Lines in the file:5000


INFO:root:Loaded data. #shape = (5000, 128)
INFO:root: #Sentences = 5000 
DEBUG:root:original_data[0][0:5] = [-2.0099535   0.08406741 -0.01919576 -1.11644971  0.14128272]
INFO:root:=========================================
INFO:root:Building model 


shape of vector space: (5000, 128)
shape of sentences: (5000, 1)


INFO:root:=========================================
INFO:root:After epoch 1, Sparsity = 51.1


...

After epoch 298, Reconstruction Loss = 1.0254, ASL = 0.0208,PSL = 0.3375, and total = 1.3162


INFO:root:After epoch 299, Sparsity = 78.3


After epoch 299, Reconstruction Loss = 1.0218, ASL = 0.0207,PSL = 0.3367, and total = 1.3118


INFO:root:After epoch 300, Sparsity = 78.4
INFO:root:Dumping the final SPine embeddings
DEBUG:root:original_data[0][0:5] = [-2.0099535   0.08406741 -0.01919576 -1.11644971  0.14128272]


After epoch 300, Reconstruction Loss = 1.0223, ASL = 0.0208,PSL = 0.3367, and total = 1.3125
shape (5000, 1000)


# Evaluating results

In [ ]:
from multiprocessing import Queue
from queue import PriorityQueue
from numpy import linalg as LA
from scipy import stats
import numpy as np
import argparse
import time
import sys
import os
from tqdm import tqdm

class Visualize:
  def __init__(self,filename):
    self.top_k_words = []
    self.zeros=0.0
    self.threshold = 0.001
    self.total = None
    self.width = 5
    self.vectors = None
    self.fileName = filename
    self.load_vectors()
  def data_read(self,line):
    
    tokens = line.strip().split("<pad2a>")
    tokens_sent=tokens[1]
    tokens_val=tokens[2]
    # tokens_sent=tokens_sent.split()
    tokens_val=tokens_val.split()
    return tokens_sent,tokens_val
  def load_vectors(self):
    
    self.vectors = {}
    self.zeros = 0.0
    f = open(self.fileName, 'r')
    lines = f.readlines()
    f.close()
    _,values=self.data_read(lines[0])
    dimension = len(values)
    self.top_k_words = [ [] for i in range(dimension)]
    c = 0
    for line in tqdm(lines):
        start = time.time()
        sentence,values=self.data_read(line)
        # sentence=sentence.replace(" ","")
        self.vectors[sentence] = [abs(float(i)) for i in values]
        self.h_dim = len(values)
        c += 1
        vector = self.vectors[sentence]
        for i, val in enumerate(vector):
          temp = self.top_k_words[i]
          if len(temp) < self.width:
            temp.append((val,sentence))
          else:
            check = temp[-1]
            if check[0] < val:
              temp[-1] = (val, sentence)
          self.top_k_words[i] = sorted(temp, reverse=True)
        self.zeros += sum([1 for i in self.vectors[sentence] if i < self.threshold])
    print ("Sparsity =", 100. * self.zeros/(len(lines)*dimension))
    self.total = len(self.vectors)
    print ('done loading vectors')
  def load_top_dimensions(self,k):
    self.top_k_words
    return
    self.dimensions = len(self.vectors[self.vectors.keys()[0]])
    for i in range(self.dimensions):
      temp = []
      while self.top_k_words[i].qsize() > 0:
        temp.append(self.top_k_words.get_nowait()[1])
      self.top_k_words[i] = temp
    print ('loaded top dimensions')
  def find_top_participating_dimensions(self,word, k):
    if word not in self.vectors:
      print ('sentence not found')
      return []
    temp = [(j, i) for i, j in enumerate(self.vectors[word])]
    answer = []
    print (" -----------------------------------------------------")
    print ("Sentence of interest = " , word)
    for i, j in sorted(temp, reverse=True)[:k]:
      print ("The contribution of the setence '%s' in dimension %d = %f" %(word, j, i))
      print ('Following are the top sentences in dimension', j, 'along with their contributions')
      print (self.top_k_words[j])
      answer.append([k[1] for k in self.top_k_words[j]])
    return

# Before Training the output

In [ ]:
# before training
bt_bertSent_embed=Visualize(filename_input)

100%|██████████| 5000/5000 [00:00<00:00, 5238.46it/s]

Sparsity = 0.09890625
done loading vectors


Following keys are avialable

In [ ]:
bt_bertSent_embed.vectors.keys()

dict_keys(['Java: Repeat Task Every Random Seconds', 'Why are Java Optionals immutable?', 'Text Overlay Image with Darkened Opacity React Native', 'Why ternary operator in swift is so picky?', 'hide/show fab with scale animation', 'Accessing pointer member of the structure using structure pointer', 'How To Disable 2nd Saturday 4th Saturday ,Sunday And Holiday Date Using JQuery Calendar', 'Resizing containers in bootstrap', 'Changing Theme in Windows 10 UWP App Programmatically', 'TextBox Value Disappears VB6', 'MongoDB Failing to Start - ***aborting after fassert() failure', "What's the best way to store text data?", 'ios/objective-c/xcode: Expand Label based on quantity of text using Storyboard', 'output FILE ,is this a fault?', 'Pod install displaying error in cocoapods version 1.0.0.beta.1', 'Haskell Stack Ghci test-suite', 'Why does the reverse() function in the Swift standard library return ReverseRandomAccessCollection?', 'eb deploy does not update the code', 'How to create a fun

In [ ]:
bt_bertSent_embed.find_top_participating_dimensions("Why are Java Optionals immutable?",3)

 -----------------------------------------------------
Sentence of interest =  Why are Java Optionals immutable?
The contribution of the setence 'Why are Java Optionals immutable?' in dimension 31 = 2.054824
Following are the top sentences in dimension 31 along with their contributions
[(3.3011715412139893, "expected primary expression before ']' token ...... line 10 column 21"), (3.1796481609344482, 'TypeScript for ... of with index / key?'), (3.1440958976745605, 'Which ad network do these guys use?'), (3.130269765853882, "When should we use '=== ' operator in javascript?"), (3.126054048538208, 'What are the platforms in the .NET Platform Standard?')]
The contribution of the setence 'Why are Java Optionals immutable?' in dimension 125 = 1.960113
Following are the top sentences in dimension 125 along with their contributions
[(3.589322328567505, 'Shared library in containers'), (3.431201934814453, 'Renaming table system unavailable SQL Azure'), (3.4136266708374023, 'Spacemacs hybrid li

# After training the output

In [ ]:
if savedAtGoogleDrive==True:
  filename_trained="/content/gdrive/MyDrive/bert_sentenceEmbedding.txt_SPINE.spine"
else:
  filename_trained="bert_sentenceEmbedding.txt_SPINE.spine"
at_bertSent_embed=Visualize(filename_trained)

100%|██████████| 5000/5000 [00:07<00:00, 700.29it/s]

Sparsity = 78.50164
done loading vectors


In [ ]:
at_bertSent_embed.find_top_participating_dimensions("Why are Java Optionals immutable?",3)

 -----------------------------------------------------
Sentence of interest =  Why are Java Optionals immutable?
The contribution of the setence 'Why are Java Optionals immutable?' in dimension 896 = 1.000000
Following are the top sentences in dimension 896 along with their contributions
[(1.0, 'Why are Java Optionals immutable?'), (1.0, 'Reference variables and pointers'), (1.0, 'How to Type Cast null as Bool in C#?'), (1.0, 'Has set been deprecated in python 2?'), (1.0, '"Django: AttributeError: NoneType object has no attribute split"')]
The contribution of the setence 'Why are Java Optionals immutable?' in dimension 688 = 1.000000
Following are the top sentences in dimension 688 along with their contributions
[(1.0, 'Why ternary operator in swift is so picky?'), (1.0, 'Why are Java Optionals immutable?'), (1.0, 'Is there a naming convention for C language implementation only (private) constants?'), (1.0, 'In Android Studio 2.0, Cannot find local variable of method in debug mode'), (